In [12]:
import pandas as pd
import ast
from tqdm import tqdm

In [13]:
df_schedule = pd.read_csv('src/access/WalkerDelta/walker_delta_access_info.csv')
# df_schedule[df_schedule['sat_id'] == 67]

In [14]:
# def find_first_last_indices(lst, target, search_idx):
#     first_index = -1  # 初始值设为 -1，表示如果找不到目标则返回 -1
#     last_index = -1  # 同上

#     # 遍历列表，寻找目标数值
#     for i, value in enumerate(lst):
#         if value == target:
#             if first_index == -1:  # 第一次找到目标值
#                 first_index = i
#             last_index = i  # 更新为最新的找到的下标

#     return first_index + search_idx, last_index + search_idx

# # # 示例列表和目标值
# # lst = [1, 2, 3, 4, 2, 5, 6, 2, 7]
# # target = 2

# # # 调用函数并打印结果
# # first_index, last_index = find_first_last_indices(lst, target)
# # print(f"First Index: {first_index}, Last Index: {last_index}")

# def find_subsequences_in_fixPeriod(df, period):
#     begin = 0
#     end = df['end_idx'].max()
#     step = int(period / 5)
#     res = []
#     count = 0
#     for search_idx in tqdm(range(begin, end, step)):
#         one_period = []
#         end_idx = min(search_idx + step, end)
#         search_list = df['sat_id'][((df['start_idx'] >= search_idx) & (df['end_idx'] <= end_idx)) | ((df['start_idx'] <= search_idx) & (df['end_idx'] > search_idx)) | ((df['start_idx'] < end_idx) & (df['end_idx'] >= end_idx))].astype(str).to_list()
#         for ch in set(search_list):
#             if(search_list.count(ch) >= 2):
#                 count += 1
#                 # 找出一个周期内出现两次的sat的在df中的index
#                 idx1, idx2 = find_first_last_indices(search_list, ch,search_idx)
#                 first_idx = df['end_idx'].iloc[idx1]
#                 second_idx = df['end_idx'].iloc[idx2]
#                 one_period.append((ch, first_idx, second_idx))
#         res.append(one_period)
#     print(count)
#     print(res)
#     return res
        
# res = find_subsequences_in_fixPeriod(df_schedule, 80 * 60)
# len(res)

In [15]:


def find_first_last_indices(lst, target):
    first_index = -1  # 初始值设为 -1，表示如果找不到目标则返回 -1
    last_index = -1  # 同上

    # 遍历列表，寻找目标数值
    for i, value in enumerate(lst):
        if value == target:
            if first_index == -1:  # 第一次找到目标值
                first_index = i
            last_index = i  # 更新为最新的找到的下标

    return first_index, last_index

# # 示例列表和目标值
# lst = [1, 2, 3, 4, 2, 5, 6, 2, 7]
# target = 2

# # 调用函数并打印结果
# first_index, last_index = find_first_last_indices(lst, target)
# print(f"First Index: {first_index}, Last Index: {last_index}")

def find_subsequences_in_fixPeriod_plus(df, period):
    stop_time = df['end_idx'].max()
    step = int(period / 5)
    res = []
    count = 0
    max_idx = 0
    begin_time = 0
    while(begin_time < stop_time):
        #  begin_time:end_time 为一个周期,找出该周期内出现至少两次的sat，并将最末的sat的end_idx作为下一个周期的起始时间，最大程度节省时间开销
        one_period = []
        end_time = min(begin_time + step, stop_time)
        # 该周期内依次出现的sat整理整列表
        search_df = df[((df['start_idx'] >= begin_time) & (df['end_idx'] <= end_time)) | ((df['start_idx'] <= begin_time) & (df['end_idx'] > begin_time)) | ((df['start_idx'] < end_time) & (df['end_idx'] >= end_time))]
        for sat_id, sat_record in search_df.groupby('sat_id'):
            if len(sat_record) >= 2:
                first_row = sat_record.iloc[0]  # 第一行
                last_row = sat_record.iloc[-1]  # 最后一行
                one_period.append((sat_id, first_row['end_idx'], last_row['start_idx'], (last_row['start_idx'] - first_row['end_idx']) * 5))
                max_idx = max(max_idx, last_row['start_idx'])
                count += 1
        begin_time = max_idx + 1
        res.append(one_period)
        print(one_period)
        if(stop_time - begin_time < step / 2):
            break
    print(res)
    return res, count / len(res)
res2,count = find_subsequences_in_fixPeriod_plus(df_schedule, 100 * 60)
len(res2)

[(0, 124, 463, 1695), (1, 3, 341, 1690), (9, 243, 587, 1720), (64, 24, 1137, 5565), (66, 922, 1110, 940), (67, 802, 994, 960), (68, 681, 880, 995), (74, 585, 1132, 2735)]
[(10, 1598, 1863, 1325), (12, 1656, 2179, 2615), (13, 1538, 2055, 2585), (14, 1419, 2309, 4450), (15, 1300, 2184, 4420), (16, 1179, 2062, 4415), (17, 1600, 1942, 1710), (64, 1148, 1348, 1000), (74, 1149, 2253, 5520)]
[(14, 2317, 3096, 3895), (21, 2901, 3419, 2590), (22, 2779, 3306, 2635), (70, 2759, 2950, 955), (71, 2643, 2832, 945), (72, 2524, 2715, 955), (73, 2402, 2601, 995), (79, 2870, 3069, 995), (86, 2649, 3204, 2775), (87, 2538, 3085, 2735), (88, 2424, 2968, 2720)]
[(20, 3568, 3906, 1690), (21, 3446, 3784, 1690), (29, 3687, 4031, 1720), (84, 3467, 4579, 5560), (86, 4365, 4553, 940), (87, 4245, 4437, 960), (88, 4123, 4323, 1000), (94, 4028, 4574, 2730)]
[(30, 5041, 5306, 1325), (32, 5100, 5622, 2610), (33, 4982, 5498, 2580), (34, 4863, 5752, 4445), (35, 4743, 5628, 4425), (36, 4623, 5506, 4415), (37, 5043, 5385,

467

In [16]:

data = []
for period in res2:
    sats = []
    start_idx = period[0][1]
    end_idx = period[0][2]
    train_time_indiv = []
    for sat_record in period:
        sats.append(sat_record[0])
        train_time_indiv.append(sat_record[-1])
        start_idx = min(start_idx, sat_record[1])
        end_idx = max(end_idx, sat_record[2])
    cost_time = (end_idx - start_idx) * 5
    data.append((sats, start_idx, end_idx, cost_time, train_time_indiv))
df_sat_avg = pd.DataFrame(data, columns=['sats','start_idx','end_idx','cost_time','train_time_indiv'])
df_sat_avg

,sats,start_idx,end_idx,cost_time,train_time_indiv
0,"[0, 1, 9, 64, 66, 67, 68, 74]",3,1137,5670,"[1695, 1690, 1720, 5565, 940, 960, 995, 2735]"
1,"[10, 12, 13, 14, 15, 16, 17, 64, 74]",1148,2309,5805,"[1325, 2615, 2585, 4450, 4420, 4415, 1710, 100..."
2,"[14, 21, 22, 70, 71, 72, 73, 79, 86, 87, 88]",2317,3419,5510,"[3895, 2590, 2635, 955, 945, 955, 995, 995, 27..."
3,"[20, 21, 29, 84, 86, 87, 88, 94]",3446,4579,5665,"[1690, 1690, 1720, 5560, 940, 960, 1000, 2730]"
4,"[30, 32, 33, 34, 35, 36, 37, 84, 94]",4591,5752,5805,"[1325, 2610, 2580, 4445, 4425, 4415, 1710, 100..."
...,...,...,...,...,...
462,"[44, 81, 83, 84, 85, 86, 87, 88]",530292,531391,5495,"[5495, 1325, 2610, 2580, 4440, 4415, 1690, 1720]"
463,"[40, 41, 42, 43, 44, 57, 58, 59, 91, 92, 93, 94]",531421,532564,5715,"[985, 955, 945, 965, 1015, 2760, 2725, 2720, 2..."
464,"[56, 57, 58, 59, 64, 65, 90, 91]",532587,533710,5615,"[955, 945, 965, 1010, 2720, 2740, 1710, 1690]"
465,"[0, 1, 3, 4, 5, 6, 7, 8, 64]",533735,534834,5495,"[1360, 1325, 2610, 2580, 4440, 4415, 1690, 172..."


In [17]:
count = 0
for res in res2:
    count += len(res)
count / len(res2)

9.967880085653105

In [18]:
res2[0],res2[1],res2[3]

([(0, 124, 463, 1695),
  (1, 3, 341, 1690),
  (9, 243, 587, 1720),
  (64, 24, 1137, 5565),
  (66, 922, 1110, 940),
  (67, 802, 994, 960),
  (68, 681, 880, 995),
  (74, 585, 1132, 2735)],
 [(10, 1598, 1863, 1325),
  (12, 1656, 2179, 2615),
  (13, 1538, 2055, 2585),
  (14, 1419, 2309, 4450),
  (15, 1300, 2184, 4420),
  (16, 1179, 2062, 4415),
  (17, 1600, 1942, 1710),
  (64, 1148, 1348, 1000),
  (74, 1149, 2253, 5520)],
 [(20, 3568, 3906, 1690),
  (21, 3446, 3784, 1690),
  (29, 3687, 4031, 1720),
  (84, 3467, 4579, 5560),
  (86, 4365, 4553, 940),
  (87, 4245, 4437, 960),
  (88, 4123, 4323, 1000),
  (94, 4028, 4574, 2730)])

In [2]:
import numpy as np
from scipy.stats import dirichlet
from scipy.special import kl_div

# 定义参数
n = 10  # 客户端数量
R = 1   # 圆环半径
alpha_0 = 0.1  # 初始浓度参数

# 计算客户端位置（角度）
theta = np.linspace(0, 2 * np.pi, n, endpoint=False)

# 计算客户端间的直线距离
D = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        D[i, j] = 2 * R * np.sin(abs(theta[i] - theta[j]) / 2)

# 生成每个客户端的狄利克雷分布参数
alphas = np.zeros((n, n))
for i in range(n):
    alphas[i] = alpha_0 * np.exp(-D[i])

# 生成每个客户端的类别比例数据
data = np.array([dirichlet(alpha).rvs()[0] for alpha in alphas])

# 计算KL散度矩阵
KL = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        KL[i, j] = np.sum(kl_div(data[i], data[j]))

data, KL  # 输出数据和KL散度矩阵


(array([[1.25274843e-03, 1.34124194e-03, 5.81358742e-01, 4.31820665e-05,
         2.19808555e-03, 5.16247155e-11, 9.57256110e-09, 1.56257084e-22,
         1.89761186e-13, 4.13805990e-01],
        [6.19264899e-10, 1.80216453e-03, 3.13279768e-09, 1.70753436e-08,
         4.18030062e-45, 1.09710443e-30, 8.28967108e-26, 9.98197497e-01,
         3.17758724e-07, 1.09116274e-36],
        [7.81257981e-08, 9.99830206e-01, 2.47745881e-07, 1.63448668e-06,
         8.30174045e-27, 7.54343049e-05, 9.23992024e-05, 4.90724927e-10,
         8.18443154e-13, 5.16857901e-11],
        [6.09212443e-02, 6.96235189e-01, 5.14775126e-07, 2.42601889e-01,
         1.89867386e-04, 3.91443070e-12, 8.90347442e-12, 4.79457254e-05,
         3.34984117e-06, 2.39267758e-19],
        [3.21575543e-04, 9.68092248e-01, 1.53668819e-26, 2.32999495e-04,
         1.72573707e-02, 1.35217796e-02, 1.22276655e-16, 5.74026305e-04,
         1.05474613e-37, 2.91499529e-14],
        [5.65577706e-37, 9.33192319e-01, 6.53731142e-20, 1.4

In [3]:
data[0]

array([1.25274843e-03, 1.34124194e-03, 5.81358742e-01, 4.31820665e-05,
       2.19808555e-03, 5.16247155e-11, 9.57256110e-09, 1.56257084e-22,
       1.89761186e-13, 4.13805990e-01])

In [13]:
for x,y in zip(data[1], data[2]):
    print(x , y)

6.192648990619384e-10 7.812579809049886e-08
0.0018021645285838024 0.9998302055911094
3.1327976810498173e-09 2.477458805064257e-07
1.7075343630435726e-08 1.6344866769408757e-06
4.180300619250363e-45 8.301740449029124e-27
1.0971044258923925e-30 7.543430493840051e-05
8.289671079248003e-26 9.239920236740913e-05
0.9981974968852859 4.907249265099513e-10
3.177587240584925e-07 8.184431540145632e-13
1.091162743549301e-36 5.168579012083426e-11


In [8]:
data[1][2]

3.1327976810498173e-09

In [12]:
float(-7.750653319143692e-08)

-7.750653319143692e-08

In [16]:
import numpy as np
from scipy.stats import dirichlet
from scipy.special import kl_div

def js_divergence(p, q):
    """计算两个分布之间的JS散度"""
    m = 0.5 * (p + q)
    return 0.5 * np.sum(kl_div(p, m)) + 0.5 * np.sum(kl_div(q, m))

# 定义参数
n = 10  # 客户端数量
R = 1   # 圆环半径
alpha_0 = 0.1  # 初始浓度参数

# 计算客户端位置（角度）
theta = np.linspace(0, 2 * np.pi, n, endpoint=False)

# 计算客户端间的直线距离
D = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        D[i, j] = 2 * R * np.sin(abs(theta[i] - theta[j]) / 2)

# 生成每个客户端的狄利克雷分布参数
alphas = np.zeros((n, n))
for i in range(n):
    alphas[i] = alpha_0 * np.exp(-D[i])

# 生成每个客户端的类别比例数据
data = np.array([dirichlet(alpha).rvs()[0] for alpha in alphas])

# 计算JS散度矩阵
JS = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        JS[i, j] = js_divergence(data[i], data[j])

# 输出数据和JS散度矩阵
# data, JS
for x,y in zip(data[1], data[2]):
    print(x , y)

1.7990543087196468e-17 0.0005341211133135216
2.5892559445777214e-07 3.5537413776552653e-09
5.137171197750882e-16 0.9628861632378188
0.00011402431344068675 6.601241654725022e-14
0.7164929904022644 3.6964305980965966e-48
1.1724018946505584e-13 1.704924453385247e-30
3.1956676596686647e-07 0.0001240496114141689
2.7662528087680632e-14 5.957697087176632e-07
0.28339240679178906 0.036455066713872504
5.292541330857501e-20 6.477075399931394e-14


In [20]:
import numpy as np
from scipy.stats import dirichlet, wasserstein_distance

# 定义参数
n = 10  # 客户端数量
R = 1   # 圆环半径
alpha_0 = 0.1  # 初始浓度参数

# 计算客户端位置（角度）
theta = np.linspace(0, 2 * np.pi, n, endpoint=False)

# 计算客户端间的直线距离
D = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        D[i, j] = 2 * R * np.sin(abs(theta[i] - theta[j]) / 2)

# 生成每个客户端的狄利克雷分布参数
alphas = np.zeros((n, n))
for i in range(n):
    alphas[i] = alpha_0 * np.exp(-D[i])

# 生成每个客户端的类别比例数据
data = np.array([dirichlet(alpha).rvs()[0] for alpha in alphas])

# 计算Wasserstein距离矩阵
Wasserstein = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        Wasserstein[i, j] = wasserstein_distance(data[i], data[j])

# 输出数据和Wasserstein距离矩阵
data, Wasserstein
for x,y in zip(data[1], data[2]):
    print(x , y)

0.09471866298372789 3.5173737337290387e-07
0.9035698472714083 0.0015258012710169721
0.0014078514246429587 0.043979086806931536
3.118539585781035e-06 1.2591272745408334e-19
5.362641389292028e-13 3.4225181833773174e-08
1.7598446862968543e-57 0.9544947257545469
0.0003005197800989166 2.4690635643005e-20
8.675862021319566e-72 2.461787865663222e-23
3.9572993937465065e-34 2.0456734307133975e-10
7.666550322691271e-65 3.8212567476937366e-13
